<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_5_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [49]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,57,hypertrophic scarring
3,85,tremor
4,94,anxiety


In [50]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-5.test.csv",header=None)
test.head()

,0,1
0,13,up all night
1,13,insomnia
2,93,shooting
3,133,night terrors
4,231,lucid dreamer


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [59]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.218614,4.650363,0.236132,18:07


In [60]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.516184,4.355989,0.261724,18:08


In [61]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.229664,4.319158,0.264742,18:06


In [62]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.190685,4.293106,0.266576,17:57


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [64]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.109286,6.388537,0.169231,00:28


In [65]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.186506,5.106488,0.138462,00:32


In [66]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.652108,4.795976,0.230769,01:17


In [67]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.407171,4.298459,0.292308,01:23
1,5.196076,4.225215,0.292308,01:22


In [68]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.033165,4.154416,0.246154,01:18
1,4.865227,3.983698,0.269231,01:16


In [69]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.643339,3.925351,0.269231,01:20
1,4.471633,3.879319,0.284615,01:15


In [70]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.325010,3.730257,0.330769,01:19
1,4.162176,3.733257,0.315385,01:17


In [71]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.980428,3.697608,0.323077,01:17
1,3.869753,3.659460,0.346154,01:17


In [72]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.759474,3.793554,0.323077,01:16
1,3.661391,3.659737,0.346154,01:19
2,3.479278,3.668984,0.384615,01:22
3,3.343961,3.701145,0.400000,01:19


In [73]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.199675,3.611131,0.384615,01:18
1,3.176408,3.755157,0.338462,01:20
2,2.996059,3.731213,0.392308,01:19
3,2.869660,3.727107,0.353846,01:22


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.696444,3.730043,0.361538,01:19
1,2.732049,3.912603,0.369231,01:15
2,2.598943,3.870245,0.346154,01:22
3,2.468439,3.894678,0.361538,01:21


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.352193,3.965696,0.384615,01:21
1,2.378280,4.031815,0.369231,01:15
2,2.258295,4.148442,0.353846,01:22
3,2.154931,4.070628,0.376923,01:18


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.021659,4.004061,0.384615,01:17
1,2.034063,4.141523,0.384615,01:21
2,1.965693,4.055862,0.361538,01:22
3,1.855190,4.098947,0.384615,01:19


In [77]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.788350,4.019027,0.353846,01:16
1,1.835908,4.192906,0.353846,01:22
2,1.782178,4.333985,0.346154,01:19
3,1.702471,4.313295,0.346154,01:18


In [78]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.602065,4.249868,0.384615,01:17
1,1.723990,4.504657,0.384615,01:16
2,1.669719,4.481573,0.353846,01:21
3,1.550493,4.408649,0.353846,01:21


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [81]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1099, 1787, 1599, 133, 1599, 233, 233, 249, 300, 1145, 1936, 824, 1599, 532, 550, 1373, 582, 749, 644, 744, 744, 749, 749, 1597, 1597, 1597, 974, 792, 792, 813, 1940, 1099, 1308, 195, 1278, 858, 1599, 974, 974, 974, 978, 978, 978, 978, 978, 978, 1007, 1019, 2149, 1767, 2147, 1099, 1925, 1136, 1508, 1175, 1599, 561, 1227, 1227, 1295, 1295, 1295, 93, 1340, 1599, 1925, 1265, 1599, 1507, 1507, 809, 561, 1341, 1508, 1599, 224, 1599, 1599, 418, 1227, 1599, 1599, 1356, 1597, 1597, 1599, 1373, 561, 1599, 1599, 1610, 1599, 1048, 2023, 749, 1711, 1718, 1718, 1767, 1727, 1718, 2026, 1767, 1669, 1767, 1727, 1718, 1073, 1099, 1787, 13, 1073, 1099, 1787, 1787, 1787, 923, 1815, 1815, 1973, 1877, 233, 1936, 1936, 109, 1942, 1599, 1798, 1432, 858, 1099, 1787, 1787, 189, 1599, 2103, 1529, 1787, 233, 2032, 1281, 1099]
[13, 13, 93, 133, 231, 233, 233, 249, 300, 362, 396, 418, 441, 532, 550, 582, 582, 596, 644, 744, 745, 749, 749, 791, 791, 791, 792, 792, 792, 813, 815, 818, 822, 852, 857, 858, 97

In [82]:
correct_num = 0

for i in range(0, 143):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

56
0.3916083916083916
